In [1]:
df=spark.sql("select * from default.test_json_parse")

In [3]:
df.show(truncate=False)

+---------------------------------------------------------------------+
|jdata                                                                |
+---------------------------------------------------------------------+
|{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }|
|{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }|
|{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }|
|{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }|
|{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }|
+---------------------------------------------------------------------+



In [4]:
from pyspark.sql.functions import *

In [5]:
from pyspark.sql.types import *

In [6]:
schema_car=StructType([StructField("cars", StringType(), True)])


schema_cars=StructType([StructField("car1", StringType(), True),
                       StructField("car2", StringType(), True),
                        StructField("car3", StringType(), True)
                       
                       ])

In [ ]:
#{  "cars": {    "car1":"Ford",    "car2":"BMW",    "car3":"Fiat"  } }

In [7]:
df1=df.select(from_json('jdata',schema_car).alias('cars')).\
select('cars.*').select(from_json('cars',schema_cars).alias('cars_info'))\
.select('cars_info.*')

In [8]:
df1.show()

+----+----+----+
|car1|car2|car3|
+----+----+----+
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
|Ford| BMW|Fiat|
+----+----+----+



In [ ]:
#'{"array_1": [{"a": "1", "b": "1"}],"array_2": [{"a": "2", "b": "2"}],"array_3": [{"a": "3", "b": "3"}]}'

In [9]:
df_a=spark.sql("select * from default.test_json_array_parse")
#df_a.show(truncate=False)
schema_array=StructType([StructField("array_1", StringType(), True),
                       StructField("array_2", StringType(), True),
                        StructField("array_3", StringType(), True),
                         StructField("array_4", StringType(), True)
                       
                       ])

schema_array_data=StructType([StructField("a", StringType(), True),
                       StructField("b", StringType(), True)
                       ])



In [10]:
df1_a=df_a.select(from_json('jadata',schema_array).alias('aaaa')).select('aaaa.*').\
    select(from_json('array_1',schema_array_data).alias('array_1'),\
    from_json('array_2',schema_array_data).alias('array_2'),\
    from_json('array_3',schema_array_data).alias('array_3')).\
    select(col('array_1.a').alias('array_1a'),col('array_1.b').alias('array_1b'),\
    col('array_2.a').alias('array_2a'),col('array_2.b').alias('array_2b'),\
    col('array_3.a').alias('array_3a'),col('array_3.b').alias('array_3b'))


df1_a.show()

+--------+--------+--------+--------+--------+--------+
|array_1a|array_1b|array_2a|array_2b|array_3a|array_3b|
+--------+--------+--------+--------+--------+--------+
|       1|       1|       2|       2|       3|       3|
|       1|       1|       2|       2|       3|       3|
|       1|       1|       2|       2|       3|       3|
|       1|       1|       2|       2|       3|       3|
|       1|       1|       2|       2|       3|       3|
+--------+--------+--------+--------+--------+--------+



In [12]:
jdf.show()

+--------------------+--------------------+-------+
|            jsondata|              jadata|  ndata|
+--------------------+--------------------+-------+
|{"cars": {"car1":...|{"array_1": [{"a"...|   null|
|{"cars": {"car1":...|{"array_1": [{"a"...|   null|
|{"cars": {"car1":...|{"array_1": [{"a"...|   null|
|{"cars": {"car1":...|{"array_1": [{"a"...|   null|
|{"cars": {"car1":...|{"array_1": [{"a"...|   null|
|                null|                null|testing|
|                null|                null|testing|
|                null|                null|testing|
+--------------------+--------------------+-------+



In [35]:
from pyspark import StorageLevel


jdf=spark.sql("select * from default.test_json_data_parse")

jdf.persist( StorageLevel.MEMORY_AND_DISK_2 )

jdf1=jdf.select('ndata',from_json('jsondata',schema_car).alias('cars'),\
    from_json('jadata',schema_array).alias('array_data'))
jdf.unpersist()
jdf1.persist( StorageLevel.MEMORY_AND_DISK_2 )

jdf2=jdf1.select('cars.*','array_data.*','ndata')

jdf1.unpersist()
jdf2.persist( StorageLevel.MEMORY_AND_DISK_2 )

jdf3=jdf2.select(from_json('cars',schema_cars).alias('cars_info'),\
       'ndata',from_json('array_1',schema_array_data).alias('array_1'),\
    from_json('array_2',schema_array_data).alias('array_2'),\
    from_json('array_3',schema_array_data).alias('array_3'),\
      from_json('array_4',schema_array_data).alias('array_4'))

jdf2.unpersist()
jdf3.persist( StorageLevel.MEMORY_AND_DISK_2 )



jdf4=jdf3.select('cars_info.*',col('array_1.a').alias('array_1a'),\
    col('array_1.b').alias('array_1b'),\
    col('array_2.a').alias('array_2a'),col('array_2.b').alias('array_2b'),\
    col('array_3.a').alias('array_3a'),col('array_3.b').alias('array_3b'),\
    col('array_4.a').alias('array_4a'),col('array_4.b').alias('array_4b'),'ndata')


jdf3.unpersist()
jdf4.persist( StorageLevel.MEMORY_AND_DISK_2 )
jdf4.show()

+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+-------+
|car1|car2|car3|array_1a|array_1b|array_2a|array_2b|array_3a|array_3b|array_4a|array_4b|  ndata|
+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+-------+
|Ford| BMW|Fiat|       1|       1|       2|       2|       3|       3|    null|    null|   null|
|Ford| BMW|Fiat|       1|       1|       2|       2|       3|       3|    null|    null|   null|
|Ford| BMW|Fiat|       1|       1|       2|       2|       3|       3|    null|    null|   null|
|Ford| BMW|Fiat|       1|       1|       2|       2|       3|       3|    null|    null|   null|
|Ford| BMW|Fiat|       1|       1|       2|       2|       3|       3|    null|    null|   null|
|null|null|null|    null|    null|    null|    null|    null|    null|    null|    null|testing|
|null|null|null|    null|    null|    null|    null|    null|    null|    null|    null|testing|
|null|null|null|    null|    n

In [36]:
jdf4.count()

8

In [37]:
jdf4.unpersist()

DataFrame[car1: string, car2: string, car3: string, array_1a: string, array_1b: string, array_2a: string, array_2b: string, array_3a: string, array_3b: string, array_4a: string, array_4b: string, ndata: string]

In [38]:
jdf4.count()

8